# 读取

In [1]:
import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from scipy.stats import pearsonr
import xarray as xr
import scipy.interpolate 
import metpy.calc as mpcalc
from sklearn.linear_model import LinearRegression
from mpl_toolkits.basemap import Basemap
import scipy.signal as signal
import proplot as plot
import math
%matplotlib
from scipy import integrate

Using matplotlib backend: TkAgg


In [ ]:
sada = xr.open_dataset('D:/ENSO_salinity/data/IAP_salinity/IAP_salinity anomaly.nc')
salinity1 = sada.salinity
salinity1.coords['time'] = pd.date_range('1980-1',freq='1M',periods=468)
salinity1 = salinity1.isel(depth_std=0)

sada1 = xr.open_dataset('D:/ENSO_salinity/data/epflux/GPCP_OAflux/GPCP_OAflux_epflux_anomaly_mmday.nc')
salinity2 = sada1.epflux
salinity2.coords['time'] = pd.date_range('1980-1',freq='1M',periods=468)

sada2 = xr.open_dataset('D:/ENSO_salinity/data/IAP_salinity/IAP_salinity_anomaly_tendency.nc')
salinity3 = sada2.salinity


salinity = [salinity1,salinity3,salinity2]

In [ ]:
def inter(var,depthmin,depthmax):
    vari = var.isel(depth_std=range(depthmin,depthmax))
    vart = vari.isel(depth_std=0).copy()
    vart.values = integrate.trapz(vari,vari.coords['depth_std'])
    return vart

In [2]:
#0-100m and 100-500m
sada = xr.open_dataset('D:/ENSO_salinity/data/IAP_salinity/IAP_salinity anomaly.nc')
data = sada.salinity
salinity1 = inter(data,0,12)/99#0-100
salinity2 = inter(data,11,23)/400#100-500
salinity = [salinity1,salinity2]

NameError: name 'inter' is not defined

# 数据处理 

In [3]:
for i in [0,1]:
    salinity[i] = salinity[i].fillna(99999999)
    scipy.signal.detrend(salinity[i],axis=0,type='linear', bp=0, overwrite_data=True)
    b,a = signal.butter(4,[1/108,1/18],'bandpass')
    salinity[i].values = signal.filtfilt(b, a, salinity[i],axis=0) 

NameError: name 'salinity' is not defined

# 回归 

In [5]:
ONI = xr.open_dataset('D:/ENSO_salinity/data/ONI/ONI.nc')
ONI = ONI.indexs

In [ ]:
pre_reg = []
pre_cor_sig = []
for j in [0,1]:
    for i in [-8,-4]:
        try:
            salinity_selected = salinity[j][:i]
            ONI_selected = ONI[-i:]
            lat, lon = salinity[j]['lat'], salinity[j]['lon']
            regression = np.array(salinity_selected).reshape(salinity_selected.shape[0],salinity_selected.shape[1]*salinity_selected.shape[2])
            A = np.vstack([ONI_selected, np.ones(len(ONI_selected))]).T
            pre_re = np.linalg.lstsq(A,regression,rcond=None)[0][0].reshape(len(lat),len(lon))
            pre_cor_si = f_regression(np.nan_to_num(regression), ONI_selected)[1].reshape(len(lat),len(lon))
            pre_reg.append(pre_re)
            pre_cor_sig.append(pre_cor_si)
        except:
            salinity_selected = salinity[j][:i]
            ONI_selected = ONI[-i:-1]
            lat, lon = salinity[j]['lat'], salinity[j]['lon']
            regression = np.array(salinity_selected).reshape(salinity_selected.shape[0],salinity_selected.shape[1]*salinity_selected.shape[2])
            A = np.vstack([ONI_selected, np.ones(len(ONI_selected))]).T
            pre_re = np.linalg.lstsq(A,regression,rcond=None)[0][0].reshape(len(lat),len(lon))
            pre_cor_si = f_regression(np.nan_to_num(regression), ONI_selected)[1].reshape(len(lat),len(lon))
            pre_reg.append(pre_re)
            pre_cor_sig.append(pre_cor_si)
        
        

    try:
        salinity_selected = salinity[j][:]
        ONI_selected = ONI[:]
        lat1, lon1 = salinity[j]['lat'], salinity[j]['lon']
        regression = np.array(salinity_selected).reshape(salinity_selected.shape[0],salinity_selected.shape[1]*salinity_selected.shape[2])
        A = np.vstack([ONI_selected, np.ones(len(ONI_selected))]).T
        pre_re = np.linalg.lstsq(A,regression,rcond=None)[0][0].reshape(len(lat1),len(lon1))
        pre_cor_si = f_regression(np.nan_to_num(regression), ONI_selected)[1].reshape(len(lat1),len(lon1))
        pre_reg.append(pre_re)
        pre_cor_sig.append(pre_cor_si)
    except:
        salinity_selected = salinity[j][:]
        ONI_selected = ONI[:-1]
        lat1, lon1 = salinity[j]['lat'], salinity[j]['lon']
        regression = np.array(salinity_selected).reshape(salinity_selected.shape[0],salinity_selected.shape[1]*salinity_selected.shape[2])
        A = np.vstack([ONI_selected, np.ones(len(ONI_selected))]).T
        pre_re = np.linalg.lstsq(A,regression,rcond=None)[0][0].reshape(len(lat1),len(lon1))
        pre_cor_si = f_regression(np.nan_to_num(regression), ONI_selected)[1].reshape(len(lat1),len(lon1))
        pre_reg.append(pre_re)
        pre_cor_sig.append(pre_cor_si)
        
    try:
        for i in [4,8]:
            salinity_selected = salinity[j][i:]
            ONI_selected = ONI[:-i]
            lat, lon = salinity[j]['lat'], salinity[j]['lon']
            regression = np.array(salinity_selected).reshape(salinity_selected.shape[0],salinity_selected.shape[1]*salinity_selected.shape[2])
            A = np.vstack([ONI_selected, np.ones(len(ONI_selected))]).T
            pre_re = np.linalg.lstsq(A,regression,rcond=None)[0][0].reshape(len(lat),len(lon))
            pre_cor_si = f_regression(np.nan_to_num(regression), ONI_selected)[1].reshape(len(lat),len(lon))
            pre_reg.append(pre_re)
            pre_cor_sig.append(pre_cor_si)
    except:
        for i in [4,8]:
            salinity_selected = salinity[j][i:]
            ONI_selected = ONI[1:-i]
            lat, lon = salinity[j]['lat'], salinity[j]['lon']
            regression = np.array(salinity_selected).reshape(salinity_selected.shape[0],salinity_selected.shape[1]*salinity_selected.shape[2])
            A = np.vstack([ONI_selected, np.ones(len(ONI_selected))]).T
            pre_re = np.linalg.lstsq(A,regression,rcond=None)[0][0].reshape(len(lat),len(lon))
            pre_cor_si = f_regression(np.nan_to_num(regression), ONI_selected)[1].reshape(len(lat),len(lon))
            pre_reg.append(pre_re)
            pre_cor_sig.append(pre_cor_si)

In [ ]:
import numpy.ma as ma
for i in range(10,15):
    pre_cor_sig[i] =  ma.masked_where(pre_reg[i]>1.1,pre_cor_sig[i])
    pre_cor_sig[i] =  ma.masked_where(pre_reg[i]<-4,pre_cor_sig[i])
    pre_reg[i] =  ma.masked_where(pre_reg[i]>1.1,pre_reg[i])
    pre_reg[i] =  ma.masked_where(pre_reg[i]<-4,pre_reg[i])

In [ ]:
del salinity1
del salinity2
#del salinity3
del salinity

In [ ]:
lat, lon = salinity[0]['lat'], salinity[0]['lon']

# 画图

In [ ]:
#SSSA

plot.rc.update({'fontsize':6})
fig, axs = plot.subplots(nrows=5,ncols=2,axwidth=5,aspect=(2,5),proj='robin',proj_kw={'lon_0':180},basemap=True,tight=True)
#fig, axs = plot.subplots(nrows=5,ncols=3,width_ratios=2,sharey=True,
                         #proj='robin',proj_kw={'lon_0':180},basemap=True,tight=True)
title = ['lag=-8','lag=-4','lag=0','lag=4','lag=8']
axs.format(collabels=['S100','S100-500'])
for i in range(0,5):
    #ax = axs[i]
    ax = axs[i,0]
    ax.format(title=title[i],land=True,landcolor='gray',labels=True,landzorder=4,gridlinewidth=0,latlines=30,lonlines=90,lonlabels=True,titleloc='right')
    level = plot.arange(-10e-2,10e-2,2e-2) #0-100
    filled_c = ax.contourf(lon,lat,pre_reg[i],levels=level,cmap='RdBu_r',extend='both',labels=False)
    filled_r = ax.contourf(lon,lat,pre_cor_sig[i],levels=[np.min(pre_cor_sig[i]),0.1,np.max(pre_cor_sig[i])],zorder=1,hatches=[ None,'.....'],colors="none")
    ax.grid(linestyle='None')
ax.grid(linestyle='None')
cbar = fig.colorbar(filled_c,ticks=4e-2,label='g $kg^{-1}$ $K^{-1}$',length=0.7,loc='b',col=1)

for i in range(5,10):
    #ax = axs[i]
    ax = axs[i-5,1]
    ax.format(title=title[i-10],land=True,landcolor='gray',labels=True,landzorder=4,gridlinewidth=0,latlines=90,lonlines=90,lonlabels=True,titleloc='right')
    level = plot.arange(-6e-3,6e-3,12e-4)
    filled_c1 = ax.contourf(lon,lat,-pre_reg[i],levels=level,cmap='RdBu_r',extend='both',labels=False)
    filled_r = ax.contourf(lon,lat,pre_cor_sig[i],levels=[np.min(pre_cor_sig[i]),0.1,np.max(pre_cor_sig[i])],zorder=1,hatches=[ None,'.....'],colors="none")
    ax.grid(linestyle='None')
ax.grid(linestyle='None')
cbar1 = fig.colorbar(filled_c1,ticks=24e-4,length=0.7,label='g $kg^{-1}$ $month^{-1}$ $K^{-1}$',col=2,loc='b',format='%.0e')


#cbar2 = fig.colorbar(filled_c2,ticks=0.4,label='mm $day^{-1}$ $K^{-1}$',loc='b',length=0.7,format='%.0e',col=3)

In [ ]:
for i in range(10,15):
    ax = axs[i-10,2]
    ax.format(title=title[i-10],land=True,landcolor='gray',labels=True,landzorder=4,gridlinewidth=0,
               latlines=90,lonlines=90,lonlabels=True,titleloc='right')
    #level = plot.arange(-1.0,1.0,0.2)
    #level = plot.arange(-0.0005,0.0005,0.0001) #0-100
    level = plot.arange(-1,1,0.2)
    filled_c2 = ax.contourf(lon,lat,pre_reg[i],levels=level,cmap='RdBu_r',extend='both',labels=False)
    filled_r = ax.contourf(lon,lat,pre_cor_sig[i],levels=[np.min(pre_cor_sig[i]),0.1,np.max(pre_cor_sig[i])],zorder=1,hatches=[ None,'.....'],colors="none")
    ax.grid(linestyle='None')
ax.grid(linestyle='None')

In [10]:
#SSSA

plot.rc.update({'fontsize':4})
#fig, axs = plot.subplots(nrows=2,ncols=5,axwidth=5,aspect=(2,5),proj='robin',proj_kw={'lon_0':180},basemap=True,tight=True)
fig, axs = plot.subplots(nrows=5,ncols=2,width_ratios=2,proj='robin',proj_kw={'lon_0':180},basemap=True,tight=True
                        ,)
title = ['lag=-8','lag=-4','lag=0','lag=4','lag=8']
axs.format(collabels=['Salinity 300-700m', 'Salinity 700-2000m'])
for i in range(0,5):
    #ax = axs[i]
    ax = axs[i,0]
    ax.format(title=title[i],land=True,landcolor='gray',labels=True,landzorder=4,gridlinewidth=0,
              lonlines=90,lonlabels=True,titleloc='right')
    #level = plot.arange(-5e-2,5e-2,10e-3) #0-100
    level = plot.arange(-0.05,0.05,0.01)
    filled_c = ax.contourf(lon,lat,pre_reg[i],levels=level,cmap='RdBu_r',extend='both',labels=False)
    filled_r = ax.contourf(lon,lat,pre_cor_sig[i],levels=[np.min(pre_cor_sig[i]),0.1,np.max(pre_cor_sig[i])],
                           zorder=1,hatches=[ None,'........'],colors="none")
    ax.grid(linestyle='None')
ax.grid(linestyle='None')
cbar = fig.colorbar(filled_c,ticks=0.02,format='%.0e',
                    label='g $kg^{-1}$ $K^{-1}$',length=0.9,loc='b',col=1)

for i in range(5,10):
    #ax = axs[i]
    ax = axs[i-5,1]
    ax.format(title=title[i-10],land=True,landcolor='gray',labels=True,landzorder=4,gridlinewidth=0,
              lonlines=90,lonlabels=True,titleloc='right')
    level = plot.arange(-0.01,0.01,0.002)
    filled_c1 = ax.contourf(lon,lat,pre_reg[i],levels=level,cmap='RdBu_r',extend='both',labels=False)
    filled_r = ax.contourf(lon,lat,pre_cor_sig[i],levels=[np.min(pre_cor_sig[i]),0.1,np.max(pre_cor_sig[i])],zorder=1,hatches=[ None,'.....'],colors="none")
    ax.grid(linestyle='None')
ax.grid(linestyle='None')
cbar1 = fig.colorbar(filled_c1,ticks=0.004,length=0.9,
                     label='g $kg^{-1}$ $K^{-1}$',col=2,loc='b',format='%.0e')

RuntimeError: Basemap is no longer maintained and is incompatible with matplotlib >= 3.3. Please use cartopy as your geographic plotting backend or downgrade to matplotlib <= 3.2.